# Notebook 07: Baseline Summary & Analysis

This notebook analyzes the **Safety-Transfer Hospital Benchmark v0.1 Dataset**.
It computes the final aggregate metrics for the baseline policy (Mock / Nav2).

**Metrics Reported:**
- **Safety Violation Rate (SVR)**: Mean % time in Red Zones.
- **Task Success Rate (TSR)**: % of episodes reaching goal (Mock: always 1.0 if not stuck).
- **Near-Violation Time (NVT)**: Mean time in Amber Zones.
- **Minimum Distances**: Distribution of closest approaches to Beds/People.

In [ ]:
import sys
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add src to path
sys.path.append(os.path.abspath('../src'))

from metrics.safety_evaluator import SafetyEvaluator

## 1. Load Dataset
Iterate through `data/dataset_v0.1` and process every episode log.

In [ ]:
DATASET_DIR = "../data/dataset_v0.1"
results = []

if not os.path.exists(DATASET_DIR):
    print(f"Dataset directory {DATASET_DIR} not found. Run Notebook 06 first.")
else:
    # Walk through worlds
    worlds = sorted([d for d in os.listdir(DATASET_DIR) if d.startswith("world_")])
    
    print(f"Found {len(worlds)} worlds to analyze.")
    
    for w_name in worlds:
        w_path = os.path.join(DATASET_DIR, w_name)
        
        # Load World Config
        with open(os.path.join(w_path, "objects.json"), 'r') as f:
            world_config = json.load(f)
        
        evaluator = SafetyEvaluator(world_config['objects'])
        
        # Process Episodes
        ep_dir = os.path.join(w_path, "episodes")
        if not os.path.exists(ep_dir): continue
            
        episodes = sorted([f for f in os.listdir(ep_dir) if f.endswith("_log.csv")])
        
        for ep_file in episodes:
            log_path = os.path.join(ep_dir, ep_file)
            log_df = pd.read_csv(log_path)
            
            if len(log_df) == 0: continue
                
            metrics, _ = evaluator.evaluate_episode(log_df)
            
            metrics['World'] = w_name
            metrics['Episode'] = ep_file
            results.append(metrics)

    df_res = pd.DataFrame(results)
    print(f"Processed {len(df_res)} episodes.")

## 2. Aggregate Statistics
Compute the benchmark table.

In [ ]:
if not df_res.empty:
    print("=== Benchmark v0.1 Summary ===")
    print(df_res[['SVR', 'Red_Steps', 'Amber_Moving_Steps', 'Min_Dist_Person', 'Min_Dist_Bed']].describe())
    
    mean_svr = df_res['SVR'].mean()
    std_svr = df_res['SVR'].std()
    print(f"\nMean SVR: {mean_svr*100:.2f}% ± {std_svr*100:.2f}%")
else:
    print("No data found.")

## 3. Visualizations

### 3.1 SVR Distribution
How safe is the baseline policy? (Expect high SVR for unconstrained baseline).

In [ ]:
if not df_res.empty:
    plt.figure(figsize=(10, 6))
    sns.histplot(df_res['SVR'], kde=True, bins=20)
    plt.title("Distribution of Safety Violation Rate (SVR)")
    plt.xlabel("SVR (Fraction of time in Red Zone)")
    plt.ylabel("Count")
    plt.grid(True, alpha=0.3)
    plt.show()

### 3.2 Minimum Distance Analysis
How close does the robot get to critical objects?

In [ ]:
if not df_res.empty:
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df_res[['Min_Dist_Person', 'Min_Dist_Bed']])
    plt.title("Minimum Distance to Objects")
    plt.ylabel("Distance (m)")
    
    # Add thresholds for reference (Person Crit)
    plt.axhline(y=0.5, color='r', linestyle='--', label='Person Crit (0.5m)')
    plt.axhline(y=0.4, color='b', linestyle='--', label='Bed Crit (0.4m)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    # Log scale might be useful if outliers are far, but linear is fine for safety zones
    plt.show()